# preparations

In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs 

In [2]:
url = 'http://bang.dangdang.com/books/bestsellers'

def get_html(url):
    try:
        useragent = {'User-Agent':'Mozilla/5.0'} #模拟浏览器
        rsp = rq.get(url, timeout=10, headers=useragent)
        rsp.raise_for_status() #根据状态码抛出HTTPError异常
        rsp.encoding = rsp.apparent_encoding #使得解码正确
        return rsp.text
    except:
        return "Exception with staus code:"+rsp.staus_code

content = get_html(url)

# First experiment

In [3]:
soup = bs(content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="图书畅销榜,畅销书推荐,畅销书排行榜,畅销书排行榜" name="keywords"/>
  <meta content="当当最新畅销书排行榜-畅销书推荐，提供真实、权威、可信的图书畅销榜数据，查看2014畅销书排行榜，就上DangDang.COM。" name="description"/>
  <title>
   图书畅销榜-近24小时畅销书排行榜-当当畅销图书排行榜
  </title>
  <script src="/books/js/jquery-1.4.2.min.js" type="text/javascript">
  </script>
  <script src="/books/js/common.js" type="text/javascript">
  </script>
  <script src="/books/js/popwin.js" type="text/javascript">
  </script>
  <script src="http://static.ddimg.cn/js/login/LoginWindow.js" type="text/javascript">
  </script>
  <link href="/books/css/bang_list.css" rel="stylesheet" type="text/css"/>
  <script src="/books/js/DD_belatedPNG_0.0.8a.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   DD_belatedPNG.

In [4]:
ranking = soup.find('ul', attrs={'class':'bang_list clearfix bang_list_mode'})
items = ranking.find_all('li')
item = items[0]

In [5]:
txts = [x.string for x in item.find_all('div') if x.string is not None]
txts

['1.', '活着（2017年新版）', '余华']

In [6]:
star = item.find('div', class_='star')
star = [x.string for x in star.contents[1:]]
star

['119335条评论', '107.5%推荐']

In [7]:
price = item.find('div', class_='price')
spans = [x.string for x in price.find_all('span')]
spans

['¥14.00', '¥28.00', '5.0折', '¥9.99']

从下面结果看出来，这样提取出来的格式还不一样（len不一样），所以最后直接用正则了

In [8]:
for item in items[:10]:
    all_attrs = [x.string for x in item.find_all('div') 
                 if x.string is not None]
    star = item.find('div', class_='star')
    star = [x.string for x in star.contents[1:]]
    all_attrs += star
    
    price = item.find('div', class_='price')
    spans = [x.string for x in price.find_all('span')]
    all_attrs += star
    
    print(all_attrs)

['1.', '活着（2017年新版）', '余华', '119335条评论', '107.5%推荐', '119335条评论', '107.5%推荐']
['2.', '神奇校车・桥梁书版（全20册）', '219230条评论', '99.8%推荐', '219230条评论', '99.8%推荐']
['3.', '我喜欢生命本来的样子(周国平经典散文作品集)', '周国平', '55501条评论', '99.2%推荐', '55501条评论', '99.2%推荐']
['4.', '少年读史记（套装全5册）', '张嘉骅', '130471条评论', '99.9%推荐', '130471条评论', '99.9%推荐']
['5.', '走进小古文・阅读与训练（供小学生使用）（第2版）', '5953条评论', '99.9%推荐', '5953条评论', '99.9%推荐']
['6.', '小熊和最好的爸爸（全7册）', '381944条评论', '99.2%推荐', '381944条评论', '99.2%推荐']
['7.', '295199条评论', '99.8%推荐', '295199条评论', '99.8%推荐']
['8.', '我的第一本地理启蒙书', '158919条评论', '99.9%推荐', '158919条评论', '99.9%推荐']
['9.', '追风筝的人', '1631769条评论', '99.8%推荐', '1631769条评论', '99.8%推荐']
['10.', '神奇校车・图画书版（全11册）', '672127条评论', '99.7%推荐', '672127条评论', '99.7%推荐']


# Second experiment

我也试了一下用`.children`不过明显是嵌套标签很多，提取不出来什么，所以还是用`.descendants`

In [9]:
import re
original = [x.string for x in item.descendants 
           if x.string is not None]
original

['\n',
 '10.',
 '10.',
 '\n',
 '\n',
 '神奇校车・图画书版（全11册）',
 '神奇校车・图画书版（全11册）',
 '神奇校车・图画书版（全11册）',
 '\n',
 '672127条评论',
 '672127条评论',
 '99.7%推荐',
 '99.7%推荐',
 '\n',
 '(美)',
 '乔安娜柯尔',
 '乔安娜柯尔',
 ' 著 ,(美)',
 '布鲁斯・迪根',
 '布鲁斯・迪根',
 ' 图',
 '\n',
 '2011-01-01',
 '2011-01-01',
 '\xa0',
 '贵州人民出版社',
 '贵州人民出版社',
 '\n',
 '\n',
 '\n',
 '¥66.00',
 '¥66.00',
 '\n',
 '¥132.00',
 '¥132.00',
 '\r\n            (',
 '5.0折',
 '5.0折',
 ')\r\n                    ',
 '\n',
 '\n',
 '\n',
 '加入购物车',
 '加入购物车',
 '\n',
 '收藏',
 '收藏',
 '\n',
 '\n',
 '\n']

#### get rid of the useless by testing if there exists `\n` or `\r`

In [10]:
filtered = [x for x in original if re.search('[\n\r]', x) is None]
filtered

['10.',
 '10.',
 '神奇校车・图画书版（全11册）',
 '神奇校车・图画书版（全11册）',
 '神奇校车・图画书版（全11册）',
 '672127条评论',
 '672127条评论',
 '99.7%推荐',
 '99.7%推荐',
 '(美)',
 '乔安娜柯尔',
 '乔安娜柯尔',
 ' 著 ,(美)',
 '布鲁斯・迪根',
 '布鲁斯・迪根',
 ' 图',
 '2011-01-01',
 '2011-01-01',
 '\xa0',
 '贵州人民出版社',
 '贵州人民出版社',
 '¥66.00',
 '¥66.00',
 '¥132.00',
 '¥132.00',
 '5.0折',
 '5.0折',
 '加入购物车',
 '加入购物车',
 '收藏',
 '收藏']

#### now the task is to eliminate the redundant
the following eliminates the repetition while keeping the order

In [11]:
filtered_ = sorted(set(filtered), key=filtered.index)
filtered_

['10.',
 '神奇校车・图画书版（全11册）',
 '672127条评论',
 '99.7%推荐',
 '(美)',
 '乔安娜柯尔',
 ' 著 ,(美)',
 '布鲁斯・迪根',
 ' 图',
 '2011-01-01',
 '\xa0',
 '贵州人民出版社',
 '¥66.00',
 '¥132.00',
 '5.0折',
 '加入购物车',
 '收藏']

#### organize the fields
以`\xa0`为轴提取信息

In [12]:
first = filtered_[:4]
banner = filtered_.index('\xa0')
second = filtered_[banner+1:banner+5]
authors = filtered_[4:banner-1]
authors = ''.join(authors)
attrs = first + [authors] + second
attrs

['10.',
 '神奇校车・图画书版（全11册）',
 '672127条评论',
 '99.7%推荐',
 '(美)乔安娜柯尔 著 ,(美)布鲁斯・迪根 图',
 '贵州人民出版社',
 '¥66.00',
 '¥132.00',
 '5.0折']

#### the final code
为了处理空格，作者中的空格全部换成了`\u3000`全角空格，同时截断只剩10个字符

In [13]:
def filtering(item):
    original = [x.string for x in item.descendants if x.string is not None and x.string != '...']
    filtered = [x for x in original if re.search('[\n\r]', x) is None]
    filtered_ = sorted(set(filtered), key=filtered.index)
    return filtered_

def extract(item):
    filtered_ = filtering(item)
    
    first = filtered_[:4]
    banner = filtered_.index('\xa0')
    second = filtered_[banner+1:banner+5]
    authors = filtered_[4:banner-1]
#     print(authors)
    authors = ''.join(authors)

    first[1] = first[1][:15]
    authors = authors[:10]
    re.sub(' ', '\u3000', authors)
    attrs = first + [authors] + second
    return attrs

[len(extract(x)) for x in items]

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [14]:
import re
def extract_from(items):
    all_attrs = []
    for item in items:
        all_attrs.append(extract(item))
    
    return all_attrs    

all_attrs = extract_from(items)

In [15]:
fmt = "{0[0]:^5}"+"{0[1]:{1}<15}\t"+"{0[6]:<5}"+"\t{0[7]:<5}\t"+"{0[8]:<5}\t"+"{0[4]:<15}\t"+"{0[2]:<10}\t"+"{0[3]:<5}"
print(fmt.format(['排名','书名','评论数','推荐','作者','出版社','折后价','原价','折扣'],chr(12288)))
for attr in all_attrs:
    print(fmt.format(attr, chr(12288)))#fmt.format(attr, chr(12288))

 排名  书名　　　　　　　　　　　　　	折后价  	原价   	折扣   	作者             	评论数       	推荐   
 1.  活着（2017年新版）　　　　	¥14.00	¥28.00	5.0折 	余华             	119335条评论 	107.5%推荐
 2.  神奇校车・桥梁书版（全20册）	¥75.00	¥150.00	5.0折 	乔安娜柯尔　著 布鲁     	219230条评论 	99.8%推荐
 3.  我喜欢生命本来的样子(周国平经	¥22.50	¥45.00	5.0折 	周国平            	55501条评论  	99.2%推荐
 4.  少年读史记（套装全5册）　　　	¥50.00	¥100.00	5.0折 	张嘉骅            	130471条评论 	99.9%推荐
 5.  走进小古文・阅读与训练（供小学	¥18.80	¥25.00	7.5折 	吴庆芳(丛书主编）      	5953条评论   	99.9%推荐
 6.  小熊和最好的爸爸（全7册）　　	¥17.50	¥35.00	5.0折 	（荷）丹姆  著，漆     	381944条评论 	99.2%推荐
 7.  这就是二十四节气（中国二十四节	¥50.00	¥100.00	5.0折 	高春香，邵敏 著，许     	295199条评论 	99.8%推荐
 8.  我的第一本地理启蒙书　　　　　	¥24.90	¥49.80	5.0折 	郑利强 段虹(绘)      	158919条评论 	99.9%推荐
 9.  追风筝的人　　　　　　　　　　	¥14.50	¥29.00	5.0折 	[美]卡勒德・胡赛尼     	1631769条评论	99.8%推荐
 10. 神奇校车・图画书版（全11册）	¥66.00	¥132.00	5.0折 	(美)乔安娜柯尔 著     	672127条评论 	99.7%推荐
 11. 神奇校车・动画版（全10册）　	¥40.00	¥80.00	5.0折 	乔安娜柯尔  著,布     	229182条评论 	99.6%推荐
 12. 东野圭吾：解忧杂货店　　　　　	¥27.30	¥39.50	6.9折 	(日)东野圭吾        	1345197条评论	99.9%推荐
 13. 人间失格（日